# Non-Gaussian Measurements Tutorial 2

## Overview

- This tutorial introduces non-Gaussian estimation via a factor graph solution.
- Factor graphs should be thought of fully probabilisitic interactions between variables, as modelled by the factors.
- This tutorial shows how measurement statistics do not have be unimodal bell curve (i.e. normal/Gaussian) and thereby introduce non-Gaussian behavior.
- This example will use only continuous stochastic variables with slightly more diverse measurement distribution.
- This tutorial is one dimension with only a few variables and factors, in order to get comfortable working with multi-modal beliefs.
- This example is showing one of four mechanisms by which non-Gaussian behavior can get introduced into a factor graph solution, see other tutorials for other mechanisms.
- This tutorial illustrates how algebraic relations (i.e. residual functions) between multiple stochastic variables are calculated, as well as the final posterior belief estimate, from several pieces of information.
- The tutorial implicitly shows a multi-modal uncertainty can be introduced from non-Gaussian measurements, and then transmitted through the system.
- The tutorial also illustrates consensus through an additional piece of information, which reduces all stochastic variable marginal beliefs to unimodal only beliefs.
- The ambiguous measurement data example shown in this tutorial can readily be incorporated in other use cases.
- Lastly, this tutorial will also show how graph-based automatic initialization of variables is achieved with the Caesar.jl solver.

### Signatures Used

`ContinuousScalar`, `Prior`, `LinearRelative`, `Mixture`, `Normal`, `plotKDE`, `plotDFG`, `isInitialized`, `initAll!`, `solveGraph!`

## Ambiguous Data Example

The application of this tutorial is presented in abstract from which the user is free to imagine any system of relationships:  For example, a robot driving in a one dimensional world; or a time traveler making uncertain jumps 
forwards and backwards in time.

To start, the two major mathematical packages are brought into scope.

In [ ]:
# Install if needed with: 
# import Pkg; Pkg.add(Pkg.PackageSpec(name="IncrementalInference", version="0.27"))
using IncrementalInference

In [ ]:
# suppress unnecessary printouts in the notebook
using Logging
Logging.disable_logging(Logging.Warn)

 
## Starting a 1D Factor Graph

### Variable, `ContinuousScalar`

The next step is to describe the inference problem with a graphical model with any of the existing concrete types that inherit from  `<: AbstractDFG`.
The first step is to create an empty factor graph object and start populating it with variable nodes.
The variable nodes are identified by `Symbol`s, namely `x0, x1, x2, x3`.


In [ ]:
# Start with an empty factor graph
fg = initfg()

# add the first node
addVariable!(fg, :x0, ContinuousScalar)

### Prior Factor, (Euclidean(1))

Factor graphs are bipartite graphs with `factors` that act as mathematical structure between interacting `variables`.
After adding node `x0`, a singleton factor of type `Prior` (which was defined by the user earlier) is 'connected to' variable node `x0`.
This unary factor is taken as a `Distributions.Normal` distribution with zero mean and a standard devitation of `1`.


In [ ]:
# this is unary (prior) factor and does not immediately trigger autoinit of :x0.
addFactor!(fg, [:x0], Prior(Normal(0,1)))

**Note**, this example is using just `Prior`, which is a simplification for the sake of this tutorial.  Look out for on-manifold prior factors in other code examples.

### Visualizing Graph Topology

In [ ]:
# Install if needed with: 
# import Pkg; Pkg.add("GraphPlot")
using GraphPlot

In [ ]:
DFG.plotDFG(fg)


The topological graph plot above shows the two node factor graph, one variable and one prior factor.  
This example uses graph-based automatic variable initialization which was discussed in ICRA 2022 Tutorial 1.

### Next Variable and Relative Factor

Now let's add a second variable `x1`, and connect it to `x0` with a `LinearRelative` factor.

In [ ]:
addVariable!(fg, :x1, ContinuousScalar)
# P(Z | :x1 - :x0 ) where Z ~ Normal(10,1)
addFactor!(fg, [:x0, :x1], LinearRelative(Normal(10.0,1)))

### Visualizing the Variable Probability Belief

The `RoMEPlotting.jl` package allows visualization (plotting) of the belief state over any of the variable nodes.
Remember the first time executions are slow given required code compilation, and that future versions of these package will use more precompilation to reduce first execution running cost.

In [ ]:
using RoMEPlotting

plotBelief(fg, :x0)


By forcing the initialization of `:x1` and plotting its belief estimate,

In [ ]:
initAll!(fg)
plotBelief(fg, [:x0, :x1])

the predicted influence of the `P(Z| X1 - X0) = LinearRelative(Normal(10, 1))` is shown by the red trace.
The red trace (predicted belief of `x1`) is noting more than the approximated convolution of the current marginal belief of `x0` with the conditional belief described by `P(Z | X1 - X0)`.


### Mixture Distribution on Next Relative Factor

Another `ContinuousScalar` variable `x2` is 'connected' to `x1` through a more complicated `MixtureRelative` likelihood function.

In [ ]:
addVariable!(fg, :x2, ContinuousScalar)
mmo = Mixture(LinearRelative, 
              (hypo1=Rayleigh(3), hypo2=Uniform(30,55)), 
              [0.4; 0.6])
addFactor!(fg, [:x1, :x2], mmo)

In [ ]:
using DistributedFactorGraphs
pack = packFactor(fg, getFactor(fg, :x1x2f1))
using JSON
@info JSON.json(pack)


The `mmo` variable illustrates how a near arbitrary mixture probability distribution can be used as a conditional relationship between variable nodes in the factor graph.
In this case, a 40%/60% balance of a Rayleigh and truncated Uniform distribution which acts as a multi-modal conditional belief.
Interpret carefully what a conditional belief of this nature actually means.

Following the tutorial's practical example frameworks (robot navigation or time travel), this multi-modal belief implies that moving from one of the probable locations in `x1` to a location in `x2` by some processes defined by `mmo=P(Z | X2, X1)` is uncertain to the same 40%/60% ratio.
In practical terms, collapsing (through observation of an event) the probabilistic likelihoods of the transition from `x1` to `x2` may result in the `x2` location being at either 15-20, or 40-65-ish units.
The predicted belief over `x2` is illustrated by plotting the predicted belief (green trace), after forcing initialization.

In [ ]:
initAll!(fg)
plotBelief(fg, [:x0, :x1, :x2])


## Adding Variable `x3`

Adding one more variable `x3` through another `LinearRelative(Normal(-50,1))`

In [ ]:
addVariable!(fg, :x3, ContinuousScalar)
addFactor!(fg, [:x2, :x3], LinearRelative(Normal(-50, 1)))

expands the factor graph to to four variables and four factors.
This part of the tutorial shows how a unimodal likelihood (conditional belief) can transmit the bimodal belief currently contained in `:x2`.

In [ ]:
initAll!(fg)
plotBelief(fg, [:x0, :x1, :x2, :x3])

Notice the blue trace (`x3`) is a shifted and slightly spread out version of the initialized belief on `x2`, through the convolution with the conditional belief `P(Z | X2, X3)`.

### The Last Factor


Global inference over the entire factor graph has still not occurred, and will at this stage produce roughly similar results to the predicted beliefs shown above.
Only by introducing more information into the factor graph can inference extract more precise marginal belief estimates for each of the variables.
A final piece of information added to this graph is a factor directly relating `x3` with `x0`.

In [ ]:
addFactor!(fg, [:x3, :x0], LinearRelative(Normal(40, 1)))

Pay close attention to what this last factor means in terms of the probability density traces shown in the previous figure.
The blue trace for `x3` has two major modes, one that overlaps with `x0, x1` near 0 and a second mode further to the left at -40.
The last factor introduces a shift `LinearRelative(Normal(40,1))` which essentially aligns the left most mode of `x3` back onto `x0`.

This last factor forces a mode selection through consensus.
By doing global inference, the new information obtained in `x3` will be equally propagated to `x2` where only one of the two modes will remain.

## Solve the Graph


Global inference is achieved with local computation using two function calls, as follows.


In [ ]:
tree = solveGraph!(fg)

# and visualization
plotBelief(fg, [:x0, :x1, :x2, :x3])

The resulting posterior marginal beliefs over all the system variables are:

## Conclusion

It is import to note that although this tutorial ends with all marginal beliefs having near Gaussian shape and are unimodal, that the package supports multi-modal belief estimates during both the prediction and global inference processes.
In fact, many of the same underlying inference functions are involved with the automatic initialization process and the global multi-modal iSAM inference procedure.
This concludes the ContinuousScalar tutorial

In [ ]:
# Should be deleted
using DistributedFactorGraphs
vs = ls(fg)
for v in vs
  @info packVariable(fg, getVariable(fg, v))
end
#fs = lsf(fg)